In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE

#want to save pdf fonts? then do this:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42


In [ ]:
ph_dir = '/global/cfs/cdirs/metatlas/projects/untargeted_tasks/20211117_JGI-AK-TH_AP_507288_EColiHSLs_final_QE-139_C18_USDAY63649_REUPLOAD_positive/'
ph_file = os.path.join(ph_dir,'20211117_JGI-AK-TH_AP_507288_EColiHSLs_final_QE-139_C18_USDAY63649_REUPLOAD_positive_peak-height.csv')
md_file = os.path.join(ph_dir,'20211117_JGI-AK-TH_AP_507288_EColiHSLs_final_QE-139_C18_USDAY63649_REUPLOAD_positive_metadata.tab')

In [ ]:
msql_features = pd.read_csv('MSQL-NF-f012b88e-extract_results-main.tsv',sep='\t')
msql_features = msql_features['scan'].tolist()
# msql_features

In [ ]:

def melt_dataframe(ph,md):
#     df.drop(columns=['filename','sample'],inplace=True)
    # df.fillna(0.0,inplace=True)
    feature_cols = ['feature_id','mz','rt']
    var_cols = [c for c in ph.columns if c.endswith('mzML')]
    df = ph.melt(id_vars=feature_cols,value_vars=var_cols)#,var_name='value')
    # df = df[~pd.isna(df['value'])]
    df['value'].fillna(0.0,inplace=True)
    df['value'] = df['value'].astype(float)
    df = pd.merge(df,md,left_on='variable',right_on='filename')
    df.drop(columns=['variable','filename'],inplace=True)
    df.reset_index(inplace=True,drop=True)
    return df

def calc_background(df,background='exctrl',background_ratio=3.0):
    exctrl = df[df['sampletype'].str.contains(background.lower())].groupby(['feature_id','sampletype'])['value'].max().reset_index()
    sample = df[~df['sampletype'].str.contains(background.lower())].groupby(['feature_id','sampletype'])['value'].max().reset_index()
    ratio_df = pd.merge(exctrl.add_suffix('_exctrl'),sample.add_suffix('_sample'),left_on='feature_id_exctrl',right_on='feature_id_sample')
    ratio_df['ratio'] = ratio_df['value_sample']/(1+ratio_df['value_exctrl'])
    good_features = ratio_df.loc[ratio_df['ratio']>background_ratio,'feature_id_sample'].unique()
    all_features = ratio_df['feature_id_sample'].unique()
    bad_features = list(set(all_features) - set(good_features))
    rm_count = len(all_features) - len(good_features)
    print('Please remove %d features out of %d'%(rm_count,len(all_features)))
    return good_features,bad_features


ph = pd.read_csv(ph_file)
cols = ph.columns
cols = [c.replace(' Peak height','') for c in cols]
ph.columns = cols
colsl = [c for c in cols if not 'Unnamed:' in c]
ph = ph[cols]
rename_cols = {'row ID':'feature_id','row m/z':'mz','row retention time':'rt'}
ph.rename(columns=rename_cols,inplace=True)

print(ph.shape)
ph = ph[ph['feature_id'].isin(msql_features)]
print(ph.shape)

md = pd.read_csv(md_file,sep='\t')
md.rename(columns={'ATTRIBUTE_sampletype':'sampletype'},inplace=True)
md.fillna('',inplace=True)
md['sampletype'] = md['sampletype'].astype(str)
md['sampletype'] = md['sampletype'].apply(lambda x: x.lower())
if any([e for e in  md['sampletype'].tolist() if 'exctrl' in e]):
    print(row['parent_dir'])
    md['filename'] = md['filename'].apply(lambda x: os.path.basename(x))

df = melt_dataframe(ph,md)
df = df[df['sampletype']!='qc']
good_features,bad_features = calc_background(df,background='media-control')
df = df[df['sampletype']!='media-control']
df = df[df['feature_id'].isin(good_features)]


In [ ]:
group_cols = ['feature_id', 'mz', 'rt', 'sampletype']
avg_df = df.groupby(group_cols).mean().reset_index(drop=False)
avg_df

In [ ]:
p = pd.pivot(data=avg_df,index='feature_id',columns='sampletype',values='value')
m = p.values
m = m[:]
min_nonzero = m[m>0].min()
p[p<min_nonzero] = 2*min_nonzero/3

In [ ]:
g = sns.clustermap(p.T.apply(np.log10),cmap='plasma',figsize=(29,30),metric='correlation')#,vmin=-5,vmax=5,center=0,figsize=(6,13))

In [ ]:
g.savefig('features_sample_bicluster.pdf')

In [ ]:
from scipy.cluster import hierarchy
from scipy.spatial.distance import pdist,squareform

In [ ]:
d = pdist(p.T.values,metric='correlation')
d = squareform(d)
d.shape

In [ ]:
z = hierarchy.linkage(d,method='single')

fig,ax = plt.subplots(figsize=(6,20))

dn = hierarchy.dendrogram(z, ax=ax,
                           above_threshold_color='#bcbddc',
                           orientation='right',
                         labels=p.columns)
fig.savefig('dendrogram.pdf')

# Do this to make tree in itol format

In [ ]:
import skbio
tree = skbio.TreeNode.from_linkage_matrix(z, p.columns)

tree.write('tree.newick',format='newick')

In [ ]:
# time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=3000)
results = tsne.fit_transform(p.T.values)
# print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

# If you want PCA, this is how to do that
# pca = PCA(n_components=2)
# results = pca.fit_transform(p.values)
# print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

fig,ax = plt.subplots(figsize=(15,15))
sns.scatterplot(x=results[:,0],y=results[:,1],
                hue=p.columns,s=150)
#                 palette=color_palette,s=150,ax=ax,legend=False, linewidth=0.5,edgecolor='grey',markers=marker_dict,style=bgc_df.index.get_level_values('bgc') 
#                )
ax.set_axis_off()
plt.show()
fig.savefig('tsne.pdf')